<a href="https://colab.research.google.com/github/FAhtisham/Latext-based-EnhancerGAN/blob/main/EngancerGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torch.utils.data import Dataset, DataLoader


import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import collections

# Load the Dataset

In [53]:
def read_fasta( fp):
        name, seq = None, []
        for line in fp:
            line = line.rstrip()
            if line.startswith(">"):
                if name: yield (name, ''.join(seq))
                name, seq = line, []
            else:
                seq.append(line)
        if name: yield (name, ''.join(seq))



def load_data():
        sequences = []
        # Reading FASTA file
        with open("permissive_enhancers","r") as fp:
            for name, seq in read_fasta(fp):
                sequences.append(seq)
            print("Sequences Read Succesfully !!!!")
        print("Total Raw Sequences: ",len(sequences))
        return sequences

def add_padding(seq, p_len):
  seq = seq + ("P" * p_len)
  return seq
  

def convert_seqs_to_words(sequences):
  f_sequences = []
  for i in range(len(sequences)):
    temp = ""
    str_ = sequences[i]
    j=0

    if len(str_)%3!=0:
      n = len(str_)
      while n % 3 != 0:
        n+=1
      str_= add_padding(str_,n-len(str_))

    for k in range(0,len(str_)):
      j+=1
      if  j%3==0:
        temp = temp + str_[k-2:j] + ' ' 
        #j+=3
    f_sequences.append(temp) 
  return f_sequences


In [55]:
f_sequences = convert_seqs_to_words(load_data())
print("Total Processed Sequences: ",len(f_sequences))

Sequences Read Succesfully !!!!
Total Raw Sequences:  43011
Total Processed Sequences:  43011


# Dataset Preprocessing

In [ ]:
class Nucleotides:
  def __init__(self, seqs):
    self.nuc_pairs= self.make_pairs(seqs)
    self.encoding= {w:i for i,w in enumerate(self.nuc_pairs,1)}
    self.decoding= {i:w for i,w in enumerate(self.nuc_pairs,1)}


  def make_pairs(self, seqs, clip=1):
    nuc_pairs= collections.Counter()

    for seq in seqs:
      nuc_pairs.update(seq)


class Sequences(Dataset):
  def __init__(self):


# Autoencoder Model

In [45]:
class Autoencoder(nn.Module):
  def __init__(self, nuc_pair_size, embedding_dims, e_hidden_dims, bottleneck_dims, dec_hidden_dims, dropout_size = 0.2):
    super().__init__()
    
    # define the vars over here (layers, objects)
    self.embedding= nn.Embedding(num_embeddings= nuc_pair_size, embedding_dim= embedding_dims)
    self.rnn1= nn.LSTM(input_size= embedding_dims, hidden_size= e_hidden_dims)
    self.fc1= nn.Linear(input= e_hidden_dims, out_features= bottleneck_dims)
    self.a1= nn.ReLU(True)
    self.dropout= nn.Dropout(dropout_size)

    self.fc2= nn.Linear(input= bottleneck_dims, output_features= d_hidden_dims)
    self.rnn2= nn.LSTM(input_size= dec_hidden_dims, hidden_size= d_hidden_dims)
    self.fc3= nn.Linear(input= d_hidden_dims, output_features= nuc_pair_size)



  def encoder(self, x):
    x= self.embedding(x)
    _,(hidden_states, _)= self.rnn1(x)
    lv= self.fc1(x) # latent vector
    lv= self.Dropout(lv)
    return lv


  def decoder(self, lv):
    lv= self.fc2(lv)
    output, _= self.rnn2(lv)
    logits= self.fc3(output)
    return logits
  
  def forward(self,x):
    lv= self.encoder(x)
    logits= self.decoder(lv)
    return (lv.squeeze(), logits)

# Training Loop


In [39]:
arr = torch.tensor([[1,2],[3,4],[5,6]])
arr.size()
arr = arr.permute(1,0)
arr.size()
arr

tensor([[1, 3, 5],
        [2, 4, 6]])

In [38]:
arr2 = torch.randn(3,5,2)
print(arr2.size())
print(arr2)
arr2= arr2.permute(2,0,1)
print(arr2.size())
print(arr2)

torch.Size([3, 5, 2])
tensor([[[ 0.2006,  1.8473],
         [ 0.1055, -1.1586],
         [ 0.1208,  0.3094],
         [-0.0632, -0.5114],
         [ 0.2881,  1.3463]],

        [[-0.3929,  1.1025],
         [ 1.2375,  0.4721],
         [-0.2026,  0.5434],
         [-0.2871, -0.9966],
         [-0.8792, -0.4815]],

        [[-1.4430, -0.9805],
         [ 0.2667,  0.6952],
         [-0.1730, -1.1442],
         [ 0.8840,  0.5541],
         [-0.0819, -0.5837]]])
torch.Size([2, 3, 5])
tensor([[[ 0.2006,  0.1055,  0.1208, -0.0632,  0.2881],
         [-0.3929,  1.2375, -0.2026, -0.2871, -0.8792],
         [-1.4430,  0.2667, -0.1730,  0.8840, -0.0819]],

        [[ 1.8473, -1.1586,  0.3094, -0.5114,  1.3463],
         [ 1.1025,  0.4721,  0.5434, -0.9966, -0.4815],
         [-0.9805,  0.6952, -1.1442,  0.5541, -0.5837]]])
